このノートブックではMLFlow trackingの使い方を確認します.

In [1]:
from typing import Any, Dict, Tuple, List
import category_encoders as ce
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from pandas import DataFrame
import optuna
import mlflow

df = sns.load_dataset('titanic')
df.head()

# 必要な特徴量を抽出
feature_names = [
    'class',
    'sex',
    'age',
    'sibsp',
    'parch',
    'fare',
    'embark_town',
    'deck',
]
df_x = df[feature_names]
df_y = df['survived']

class IntOrdEncoder(ce.OrdinalEncoder):
    def __init__(self, cols, mapping, handle_unknown):
        super().__init__(cols=cols, mapping=mapping, handle_unknown=handle_unknown)
        self.cols = cols

    def transform(self, *args, **kwargs):
        """xはpd.DataFrameです.
        """
        x = super().transform(*args, **kwargs)
        for col in self.cols:
            x[col] = x[col].astype(int)

        return x

    def fit_transform(self, *args, **kwargs):
        """xはpd.DataFrameです.
        """
        x = super().fit_transform(*args, **kwargs)
        for col in self.cols:
            x[col] = x[col].astype(int)

        return x


x_tr, x_te, y_tr, y_te = train_test_split(df_x, df_y, test_size=0.33, shuffle=True, random_state=42)


def titanic_cat_encoder() -> Tuple[List[str], ce.OrdinalEncoder]:
    cols = ['class', 'sex', 'embark_town', 'deck']
    mapping = [
        {"col": "class", "mapping": {"First": 0, "Second": 1, "Third": 2}},
        {"col": "sex", "mapping": {"male": 0, "female": 1}},
        {"col": "embark_town", "mapping": {"Southampton": 0, "Cherbourg": 1, "Queenstown": 2}},
        {"col": "deck", "mapping": {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}},
    ]
    return (cols, IntOrdEncoder(cols=cols, mapping=mapping, handle_unknown='value'))

/Users/taku-y/github/PyDataOsaka/learn-mlflow/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MLFlow trackingは1回の実験をRunとして扱います. 
Runの単位は明記されていませんが, 教師あり学習の多くの例ではモデルの訓練と評価を1つのRunとします.
MLFlowはRunの様々な情報を記録することができます.
記録する値は複数の種類に分類され, それぞれ専用の関数が用意されています.
代表的なものがパラメータとメトリクスで, それぞれ`log_param()`と`log_metric()`で記録できます.

最も簡単な方法では, Runはコンテキストマネージャで管理されます.
また, Runはネストすることが可能で, 以下の例では複数のハイパーパラメータに対する訓練と評価をまとめて1つのRunとして扱います.
そのために, 引数`nested`を用いて訓練と評価のRunを実行する関数を定義します.
この関数の中で下位のRunが実行されます.

In [2]:
def fit_eval(x, y, encoder, params, cols, nested):
    with mlflow.start_run(nested=nested):
        mlflow.log_param("depth", params["depth"])
        mlflow.log_param("learning_rate", params["learning_rate"])
        mlflow.log_param("random_strength", params["random_strength"])
        mlflow.log_param("l2_leaf_reg", params["l2_leaf_reg"])

        # # cat_featuresを正しい値に設定
        # cat_features = None if params["cat_features"] == "none" else cols
        # params["cat_features"] = cat_features
        params["cat_features"] = ["embark_town", "deck"]

        # パイプラインを構成
        clf = CatBoostClassifier(
            **params,
            verbose=False
        )
        pipe = make_pipeline(encoder, clf)

        # スコアを計算
        score = cross_val_score(pipe, x, y, cv=5).mean()
        mlflow.log_metric("cv_score", score)

        return score


上位のRunをコンテキストマネージャで開始し, 
上で定義した関数を4つのハイパーパラメータの組に対して呼び出します.

In [3]:

paramss = [
  {"depth": 1, "learning_rate": 1e-2, "random_strength": 2, "l2_leaf_reg": 1.2},
  {"depth": 3, "learning_rate": 1e-3, "random_strength": 7, "l2_leaf_reg": 9.1},
  {"depth": 6, "learning_rate": 1e-2, "random_strength": 1, "l2_leaf_reg": 3.4},
  {"depth": 2, "learning_rate": 1e-3, "random_strength": 8, "l2_leaf_reg": 6.2},
]

with mlflow.start_run():
    (cols, encoder) = titanic_cat_encoder()

    for params in paramss:
        fit_eval(x_tr, y_tr, encoder, params, cols, nested=True)


実行が完了すると, `mlruns`ディレクトリの中に結果が記録されます.
結果の記録先を指定しなかったのでディレクトリの中に保存されました.
記録先にはいくつかのオプションがありますが, ここでは説明は割愛します.

`mlruns`が存在するディレクトリで`mlflow ui`を実行すると結果を確認できるUIサーバが起動します.
このサーバには`localhost:5000`でブラウザ上からアクセスできます.

![](mlflow-tracking.png)